In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df1 = pd.read_csv('user_ratings.data', sep='\t', names=column_names)
df1.head(3)

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949


In [4]:
#len(df1)

In [5]:
movie_titles = pd.read_csv("id_titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
len(df1), len(movie_titles)

(100003, 1682)

In [7]:
df1.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


- merge on item_id using pd.merge() on item_id

In [8]:
df = pd.merge(movie_titles, df1, on='item_id')

In [9]:
df.head()

,item_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324


- ratings dataframe by taking a groupby on ratings and mean()

In [11]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())

In [13]:
ratings.head(3)

,rating
title,
'Til There Was You (1997),2.333333
1-900 (1994),2.600000
101 Dalmatians (1996),2.908257


In [15]:
ratings['count'] = df.groupby('title')['rating'].count()

In [16]:
ratings.head(4)

,rating,count
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125


In [46]:
ratings.sort_values('count', ascending=False).head(10)

,rating,count
title,,
Star Wars (1977),4.359589,584
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485
"English Patient, The (1996)",3.656965,481
Scream (1996),3.441423,478
Toy Story (1995),3.878319,452
Air Force One (1997),3.631090,431


- create a pivot table df.pivot_table(index='user_id',columns='title',values='rating')

In [22]:
df.head(3)

,item_id,title,user_id,rating,timestamp
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411


In [23]:
movie_matrix= df.pivot_table(index='user_id', columns='title', values='rating')

In [27]:
len(movie_matrix)

944

In [24]:
movie_matrix.head(3)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- get data from the pivot table for a movie

In [49]:
def print_reco(movie_name):
    movie = movie_matrix[movie_name]
    movie_corr = pd.DataFrame(movie_matrix.corrwith(movie),columns=['corr'])
    movie_corr_count = movie_corr.join(ratings)
    movie_corr_count = movie_corr_count[movie_corr_count['count'] > 100]
    print(movie_corr_count.sort_values('corr', ascending=False).head(20)[['corr']])


In [51]:
print_reco('Star Wars (1977)')

D:\sw\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
D:\sw\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


                                                        corr
title                                                       
Star Wars (1977)                                    1.000000
Empire Strikes Back, The (1980)                     0.748353
Return of the Jedi (1983)                           0.672556
Raiders of the Lost Ark (1981)                      0.536117
Austin Powers: International Man of Mystery (1997)  0.377433
Sting, The (1973)                                   0.367538
Indiana Jones and the Last Crusade (1989)           0.350107
Pinocchio (1940)                                    0.347868
Frighteners, The (1996)                             0.332729
L.A. Confidential (1997)                            0.319065
Wag the Dog (1997)                                  0.318645
Dumbo (1941)                                        0.317656
Bridge on the River Kwai, The (1957)                0.316580
Philadelphia Story, The (1940)                      0.314272
Miracle on 34th Street (

In [25]:
star_wars = movie_matrix['Star Wars (1977)']

- use corrwith on the matrix and pass the data of the movie

In [29]:
star_wars_corr=pd.DataFrame(movie_matrix.corrwith(star_wars), columns=['corr'])

D:\sw\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
D:\sw\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


In [36]:
star_wars_corr_count = star_wars_corr.join(ratings)

In [40]:
star_wars_corr_count = star_wars_corr_count[star_wars_corr_count['count'] > 100]

In [41]:
star_wars_corr_count.sort_values('corr', ascending=False).head(10)

,corr,rating,count
title,,,
Star Wars (1977),1.000000,4.359589,584
"Empire Strikes Back, The (1980)",0.748353,4.206522,368
Return of the Jedi (1983),0.672556,4.007890,507
Raiders of the Lost Ark (1981),0.536117,4.252381,420
Austin Powers: International Man of Mystery (1997),0.377433,3.246154,130
"Sting, The (1973)",0.367538,4.058091,241
Indiana Jones and the Last Crusade (1989),0.350107,3.930514,331
Pinocchio (1940),0.347868,3.673267,101
"Frighteners, The (1996)",0.332729,3.234783,115
